In [1]:
import numpy as np
import matplotlib.pyplot as plt

# RNA Folding using Zucker minimization algorithm

We want to find all optimal secondary structures of a given **RNA sequence**, **AAUACUCCGUUGCAGCAU** using a simplified Zuker minimization algorithm. 

In [17]:
rna_sequence = 'AAUACUCCGUUGCAGCAU'

We initialize the $W$ and $V$ matrices, in order to begin with the implementation of the simplified **Zuker minimization algorithm**.

We initialize ou algorithm as follows:

- $j +5 > i \Longrightarrow V(i,j){=} W(i,j){=} \infty, i>j,$

We want our algorthm to take into consideration the fact that no bond can be created between two bases that have a distance greater than a predifined threshold (in our case 5).

In [18]:
# Initialization of W and V
n = len(rna_sequence)

W = np.zeros((n,n))
V = np.zeros((n,n))
traceback = np.zeros((n,n))

const = 5

for i in range(n): # columns
    for j in range(n): # rows
        if i - const < j:
            W[i, j] = np.inf
            V[i, j] = np.inf
        else:
            W[i, j] = np.nan
            V[i, j] = np.nan         
            
for i in range(n):
    print(W.T[i,:])

[inf inf inf inf inf nan nan nan nan nan nan nan nan nan nan nan nan nan]
[inf inf inf inf inf inf nan nan nan nan nan nan nan nan nan nan nan nan]
[inf inf inf inf inf inf inf nan nan nan nan nan nan nan nan nan nan nan]
[inf inf inf inf inf inf inf inf nan nan nan nan nan nan nan nan nan nan]
[inf inf inf inf inf inf inf inf inf nan nan nan nan nan nan nan nan nan]
[inf inf inf inf inf inf inf inf inf inf nan nan nan nan nan nan nan nan]
[inf inf inf inf inf inf inf inf inf inf inf nan nan nan nan nan nan nan]
[inf inf inf inf inf inf inf inf inf inf inf inf nan nan nan nan nan nan]
[inf inf inf inf inf inf inf inf inf inf inf inf inf nan nan nan nan nan]
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf nan nan nan nan]
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf nan nan nan]
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf nan nan]
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf nan]
[inf inf inf inf inf inf inf inf inf i

Moreover we define the hairpin energy as follows:

-$h(i,j)=2(i-j+5)$

In [19]:
def h(i, j):
    return 2 * (i - j + const)

and the stem energy as follows:

- $s(i,j)=-4, 0, 4$ if we have a Watson-Crick bond, a GU bond, and for all the others respectively.

As Watson-Crick bonds are considered the pairs A-U, U-A, G-C and C-G.

In [20]:
def s(i, j):
    # Watson-Crick bonds
    if (rna_sequence[i] == 'A' and rna_sequence[j] == 'U') or \
    (rna_sequence[i] == 'U' and rna_sequence[j] == 'A') or \
    (rna_sequence[i] == 'G' and rna_sequence[j] == 'C') or \
    (rna_sequence[i] == 'C' and rna_sequence[j] == 'G'):
        
        return -4
    # Pyr G-U bonds
    elif (rna_sequence[i] == 'G' and rna_sequence[j] == 'U') or \
    (rna_sequence[i] == 'U' and rna_sequence[j] == 'G'):
        
        return 0
    # Other    
    else:
        return 4

We create the `compute_min_W()` function, that was implemented recursively, to calculate the value of each cell of the $W$ matrix according to the following: 

$$ W (i, j) = min \begin{cases}
W (i − 1, j), \\
W (i, j + 1), \\
V (i, j), (i, j) defines structural element, \\
min k {W (i, k) + W (k − 1, j) : j + 1 < k < i} \\
\end{cases} $$

In [21]:
def compute_min_W(i, j):
    
    # Compute the minimum V(i,j)
    v_char = compute_min_V(i, j)
    
    # Compute the minimum W(i,k) + W(k-1, j)
    min_k = 0
    min_k_value = np.inf
    for k in range(j + 2, i):
        curr = W[i, k] + W[k-1, j]
        if curr < min_k_value:
            min_k = k
            min_k_value = curr
    

    # Find the minimum W(i,j) adn fill the block
    W[i, j] = min(W[i-1, j], W[i, j+1], V[i, j], min_k_value)
    
    
    # Traceback
    if W[i, j] == W[i-1, j]:
        traceback[i, j] = ord('l')
    elif W[i, j] == W[i, j+1]:
        traceback[i, j] = ord('d')
    elif W[i, j] == V[i, j]:
        traceback[i, j] = ord(v_char)
    else: 
        traceback[i, j] = min_k 

We create the `compute_min_V()` function, that was also implemented recursively, to calculate the value of each cell of the $V$ matrix according to the following: 

$$ V (i, j) = min \begin{cases}
s(i, j) + h(i − 1, j + 1), \\
s(i, j) + W (i − 1, j + 1), \\
\end{cases} $$

The $s(i, j) + h(i − 1, j + 1)$ term corresponds to the hairpins, and defines the unmatched $r_{j+1}, \dots, r_{i-1}$) and the $s(i, j) + W (i − 1, j + 1)$ corresponds to the matches.

In [22]:
def compute_min_V(i,j):
    
    hairpin = s(i, j) + h(i-1, j+1)
    diagonal = s(i, j) + W[i-1, j+1]
    
    V[i, j] = min(hairpin, diagonal)
    
    return 'h' if V[i,j] == hairpin else 'g'

In both `compute_min_W()` and `compute_min_V()` apart from filling the columns and raws of the $W$ and $V$ matrices, we also hold information in the `traceback` matrix, regarding which of the the four cases contributed in the calculation of the $W(i,j)$ value. 

In this way, we can extract all the information needed in order to find the backtrack path that leads to the optimal secondary conformation of the given RNA structure.

In the next step we fill in columns $i = 6, . . . , n $, rows $j = 1, . . . , n − 5$; from diagonal up, right by calling the `compute_min_W`.

In [23]:
cur_i = const
cur_j = 0
step = 1

while(step <= n - const):
    
    compute_min_W(cur_i, cur_j)
    
    
    cur_i += 1
    cur_j += 1
    
    if (cur_i == n):
        cur_i = const + step
        cur_j = 0
        step += 1
              
W = W.T
V = V.T
traceback = traceback.T

Below is presented the filled $W$ matrix, and as expected the overall minimum energy is at the top right position, $(i = n, j = 1)$.

In [24]:
print('The computed W matrix: \n')
for i in range(n):
    print(W[i,:])

The computed W matrix: 

[inf inf inf inf inf 12. 12. 12. 12. 12. 10. 10. 10.  8.  4.  4.  0. -4.]
[inf inf inf inf inf inf 20. 20. 18. 14. 10. 10. 10.  8.  4.  4.  0. -4.]
[inf inf inf inf inf inf inf 20. 18. 14. 14. 12. 12.  8.  4.  4.  0.  0.]
[inf inf inf inf inf inf inf inf 20. 14. 14. 12. 12.  8.  4.  4.  4.  0.]
[inf inf inf inf inf inf inf inf inf 20. 20. 12. 12.  8.  4.  4.  4.  4.]
[inf inf inf inf inf inf inf inf inf inf 20. 12. 12.  8.  8.  8.  8.  8.]
[inf inf inf inf inf inf inf inf inf inf inf 12. 12. 12. 12. 12. 12. 12.]
[inf inf inf inf inf inf inf inf inf inf inf inf 20. 20. 16. 12. 12. 12.]
[inf inf inf inf inf inf inf inf inf inf inf inf inf 20. 16. 12. 12. 12.]
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf 16. 16. 14. 14.]
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf 20. 14. 14.]
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf 20. 18.]
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf 20.]
[inf inf inf 

And the filled $V$ matrix.

In [25]:
print('The computed V matrix: \n')
for i in range(n):
    print(V[i,:])

The computed V matrix: 

[inf inf inf inf inf 12. 22. 24. 24. 14. 10. 14. 14. 14. 12.  8.  8. -4.]
[inf inf inf inf inf inf 20. 22. 24. 14. 10. 18. 16. 16. 12.  8.  8. -4.]
[inf inf inf inf inf inf inf 20. 18. 24. 18. 14. 16.  8.  8.  8.  0.  8.]
[inf inf inf inf inf inf inf inf 20. 14. 16. 24. 16. 16. 12.  8.  8.  0.]
[inf inf inf inf inf inf inf inf inf 20. 22. 16. 16. 16.  4. 12. 12. 12.]
[inf inf inf inf inf inf inf inf inf inf 20. 18. 16.  8. 12. 16.  8. 16.]
[inf inf inf inf inf inf inf inf inf inf inf 12. 22. 24. 16. 20. 16. 16.]
[inf inf inf inf inf inf inf inf inf inf inf inf 20. 22. 16. 20. 16. 16.]
[inf inf inf inf inf inf inf inf inf inf inf inf inf 20. 22. 12. 20. 14.]
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf 16. 22. 16. 18.]
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf 20. 14. 24.]
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf 20. 18.]
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf 20.]
[inf inf inf 

Next, we created a function for performing backtracking, in order to trace the origin of the resulted value in a given position $i, j$ of the $W$ matrix. The function recursively traces the path for the given position in $W$ using the traceback matrix populated by the `compute_min_W` function. Moreover, the function stores the positions of $W$ that reflect the hairpins and the matches of the bases found along the way to the given $i, j$ position.

In [26]:
def backtrack(i, j):
    pairs = []
    path = []    
    def helper(i, j, matches):
        if traceback[i,j] == 0:
            return
        elif traceback[i,j] == ord('l'):
            path.append('left')
            helper(i, j-1, matches)
        elif traceback[i,j] == ord('d'):
            path.append('down')
            helper(i+1, j, matches)
        elif traceback[i,j] == ord('h'):
            matches.append((i,j))
            print('Matches: ' + str(matches))
        else: 
            path.append('diagonal')
#             print(chr(int(traceback[i,j])))
            matches.append((i,j))
            helper(i+1, j-1, matches)
    helper(i, j, pairs)
    return pairs , path

We run the `backtrack()` function for the position $1,n$, and we find the path that traces the steps for the calculation of this position. 

We also find that the *matched pairs*, the bonds are formed  between:

- (2,18) : A-U
- (3,17) : U-A
- (5,15) : C-G
- (6,14) : U-A

and a *hairpin* is formed from the bond between:

- (7,12) : C-G.

In [27]:
pairs, path = backtrack(0, n-1)

rna_sequence_struct = ['.' for _ in rna_sequence]
for i, j in pairs:
    rna_sequence_struct[i] = '('
    rna_sequence_struct[j] = ')'
rna_sequence_struct = ''.join(rna_sequence_struct)    
    
print('The backtrack path: '+ str(path))

Matches: [(1, 17), (2, 16), (4, 14), (5, 13), (6, 11)]
The backtrack path: ['down', 'diagonal', 'diagonal', 'left', 'down', 'diagonal', 'diagonal', 'left']


In [28]:
rna_sequence_struct

'.((.(((....).)).))'

The following function (`draw_structure`) is needed in order to draw the secondary structure of RNA. We use the **draw_rna** project (https://github.com/DasLab/draw_rna), provided by *DasLab* (https://daslab.stanford.edu/). 

The `draw_structure` function, has two basic arguments:

1. the rna sequence as a string, in our case **rna ='AAUACUCCGUUGCAGCAU'**, and
2. a sequence using the notation **.**, **(**, **)**, to represent the absence of a bond, and the first, second base that created a bond, respectively. In our case **rna_notation_structure = `'.((.(((....).)).))'`**. 



In [14]:
def draw_struct(seq, secstruct, c=None, line=False, large_mode=False,
 cmap='viridis', rotation=0, vmin=None, vmax=None, alpha=None, ax=None):
    '''
    Draw sequence with secondary structure.
    Inputs:
    c (string or array-like).  If string, characters must correspond to colors.
     If array-like obj, used as mapping for colormap (provided in cmap), or a string.
    line (bool): draw secstruct as line.
    large_mode: draw outer loop as straight line.
    rotation: rotate molecule (in degrees).
    '''

    if c is not None:
        assert len(c) == len(seq)
        if isinstance(c[0], float):
            d.draw_rna(seq, secstruct, c, line=line, ext_color_file=True, cmap_name = cmap, vmin=vmin, vmax=vmax,
             rotation=rotation, large_mode = large_mode, alpha=alpha, ax=ax)
        else:
            d.draw_rna(seq, secstruct, c,  line=line, cmap_name=cmap, large_mode=large_mode, vmin=vmin, vmax=vmax,
             rotation=rotation, alpha=alpha, ax=ax)

    else:
        d.draw_rna(seq, secstruct, seq2col(seq), line=line, cmap_name = cmap, vmin=vmin, vmax=vmax,
         large_mode = large_mode, rotation=rotation, alpha=alpha, ax=ax)

    if ax is None:
        plt.show()

In [15]:
print('"2D RNA structure"')

rna2d = draw_struct(rna_sequence, ''.join(rna_sequence_struct))

"2D RNA structure"


NameError: name 'd' is not defined